In [1]:
import random
import PyTST.CPM

The module PyTST.CPM provides the classes Interface and CallbackWrapper.
To initialize the model, run:

In [2]:
cpm = PyTST.CPM.Interface("chemotaxis.par")  # initialize TST with chemotaxis.par

Using the method run(its), we run the model for its Monte Carlo steps:

In [3]:
cpm.run(1000)

This runs vanilla TST. To modify the algorithm one should subclass CallbackWrapper. This wrapper provides the following hooks:

init(sigma):
This method gets called once after initialization of Dish in TST.
sigma is the initialized spin matrix

copy_attempt(sigma,x,y,xp,yp,ext_DH,s):
Called from the main loop of AmoebaeMove in TST. First TST computes DH as usual. The resulting DH is passed as ext_DH, with s the connectivity dissipation.
Return True when the copy attempt gets accepted.

timestep(sigma):
Called each Monte Carlo timestep.

Furthermore, CallbackWrapper provides the method copy_prob(DH,s), which computes the acceptance probability as usual.


An example of a typical decoration of the subclass:

In [5]:
class cbWrap(PyTST.CPM.CallbackWrapper):
    def init(self,sigma):
        # called upon initialization of CPM code
        pass

    def copy_attempt(self,sigma,x,y,xp,yp,ext_DH,stiffness):
        # determines whether copy attempt gets accepted
        DH = ext_DH + self.DeltaH(x,y,xp,yp)
        copy_accepted = self.copy_accept(DH,stiffness)
        if copy_accepted:
            self.commit_move(x,y,xp,yp)
        return copy_accepted

    def copy_accept(self,DH,s):
        # decide whehter copy gets accepted based on energy differential
        # return True when accepted
        p = self.copy_prob(DH,s) # self.copy_prob is provided by libTST
        s = random.uniform(0,1)
        return s <= p

    def DeltaH(self,x,y,xp,yp):
        # determines virtual work
        return 0

    def commit_move(self,x,y,xp,yp):
        # called whenever copy attempt has been accepted
        pass

    def timestep(self,sigma):
        # called each Monte Carlo timestep
        pass

To load the wrapper, run:

In [5]:
cb = cbWrap()
cpm.set_callback_wrapper(cb)